Notebook notes: this notebook is used to take the trained model and evaluate a directory with tiff images and then save the results into a pickle file

In [22]:
%load_ext autoreload
%autoreload 2
import glob
import rasterio
import numpy as np
import pickle
import torch
import re
import sys
sys.path.insert(0, '../models')
from unet_model import UNet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Get directories and list of tif file names from 2020
data_dir = "../../../notebooks/home/data/raw/planet_images_il-2020/"
file_list = glob.glob(data_dir + "*.tif")
segDict = {}

In [19]:
# Load model
model_pretrained = UNet(3, 2)
model_pretrained.load_state_dict(torch.load('../../../saved_models/finished/model85.pth'))

<All keys matched successfully>

In [15]:
image_name = re.search(r'(?<=il-2020/)[A-z0-9\-]+', file).group()

In [53]:
for file in file_list:
    image_name = re.search(r'(?<=il-2020/)[A-z0-9\-]+', file).group()
    segDict[image_name] = {}
    # Load rasterio image
    with rasterio.open(file) as src:
        b, g, r, n = src.read()
    rgb = np.stack((r,g,b), axis=0) 

    # Calculate mask and save as dictionary
    if np.prod(rgb[0].shape) < 400*400:    
        image_array = torch.Tensor(((rgb/rgb.max())*255).astype(np.uint8)).unsqueeze(0)
        outputs = model_pretrained(image_array)
        _, predictions = torch.max(outputs, 1)
        predictions_one_hot = torch.nn.functional.one_hot(predictions).permute(0, 3, 1, 2).squeeze(0)        
        segDict[image_name]['BACKGROUND'] = predictions_one_hot[0]
        segDict[image_name]['CAFO Shed'] = predictions_one_hot[1]

In [57]:
save_dir = "../../../notebooks/home/data/processed/segmented_masks/planet_images_il-2020_masks.p"
with open(save_dir, "wb") as f:
    pickle.dump(segDict, f)

/anaconda/envs/dssg/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
